<a href="https://colab.research.google.com/github/jessejinnaruiz/gen-ai-gans-2020/blob/main/Copy_of_TESTING_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STYLEGAN2-ADA tests with abstract paintings
##### 2021-12-24


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 26 13:50:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%tensorflow_version 1.x

!git clone https://github.com/dvschultz/stylegan2
!pip install opensimplex # needed for noise interpolation
%cd stylegan2
%mkdir datasets

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 13.29 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/stylegan2


#Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). I recommend doing this on your server because the files become quite large and will be slow to upload over FTP.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [ ]:
%pwd

'/content/stylegan2'

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

project_path= drive_path / 'My Drive' / 'StyleGAN2-ADA'
dataset_name = 'Shapes-And-Forms-Test2'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
local_dataset_path = content_path / 'dataset'
local_dataset_path_to_loc_ds = content_path / 'stylegan2' / 'datasets'

Mounted at /content/drive


In [ ]:
if not local_dataset_path.is_dir():
    print("Importing dataset...")
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file succesfuly imported")
else:
    print('Zip file already imported')
# create a new subfolder for the images
%mkdir "{local_dataset_path}/images"
# set a new variable for the path to the images folder where we can extract the images into
local_dataset_path_to_images = content_path / 'dataset' / 'images'

# use zipfile lib to extract images from the zip file
import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path_to_images))
print('Extraction completed')

Importing dataset...
Zip file succesfuly imported
Extraction completed


In [ ]:
%pwd

'/content/stylegan2'

In [ ]:
%mkdir source_imgs

In [ ]:
%cp -a ../dataset/images ./source_imgs/

In [ ]:
!python dataset_tool.py create_from_images ./datasets/TF-shapesforms2 ./source_imgs/images

Loading images from "./source_imgs/images"
  0% 0/1000 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 1000/1000 [04:37<00:00,  3.60it/s]


#Training
I do not recommend attempting to train your model on Colab. It will be a very tedious process. Here are the steps, however.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.


### First run on 12/24, used mirror-augment=true.
### Ran on 12/26, used same dataset and mirror-augment=false.

In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=TF-shapesforms2 --mirror-augment=false

Couldn't find valid snapshot, starting over
Local submit - run_dir: results/00000-stylegan2-TF-shapesforms2-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x55ab8d308000 @  0x7f403d6c1001 0x7f403a1a554f 0x7f403a1f5b58 0x7f403a1f9b17 0x7f403a298203 0x55ab85e33544 0x55ab85e33240 0x55ab85ea7627 0x55ab85ea1ced 0x55ab85e3548c 0x55ab85e76159 0x55ab85e730a4 0x55ab85e35698 0x55ab85ea3fe4 0x55ab85ea19ee 0x55ab85d73e2b 0x55ab85ea3fe4 0x55ab85ea19ee 0x55ab85d73e2b 0x55ab85ea3fe4 0x55ab85e34afa 0x55ab85ea2915 0x55ab85e34afa 0x55ab85ea2c0d 0x55ab85ea19ee 0x55ab85d73e2b 0x55ab85ea3fe4 0x55ab85ea19ee 0x55ab85d73e2b 0x55ab85ea3fe4 0x55ab85e34afa
tcmalloc: large alloc 4294967296 bytes == 0x55ac8d308000 @  0x7f403d6bf1e7 0x7f403a1a546e 0x7f403a1f5c7b 0x7f403a1f635f 0x7f403a298103 0x55ab85e33544 0x55ab85e33240 0x55ab85ea7627 0x55ab85ea19ee 0x55ab85e34bda 0x55ab85ea3737 

Once running, your training files will show up in the results folder.

#Test the model
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
!python run_generator.py generate-images --network=/content/stylegan2/results/00000-stylegan2-TF-shapesforms2-1gpu-config-f/network-snapshot-010000.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00001-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/stylegan2/results/00000-stylegan2-TF-shapesforms2-1gpu-config-f/network-snapshot-010000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 3875451 (1/550) ...
Generating image for seed 3875452 (2/550) ...
Generating image for seed 3875453 (3/550) ...
Generating image for seed 3875454 (4/550) ...
Generating image for seed 3875455 (5/550) ...
Generating image for seed 3875456 (6/550) ...
Generating image for seed 3875457 (7/550) ...
Generating image for seed 3875458 (8/550) ...
Generating image for seed 3875459 (9/550) ...
Generating image for seed 3875460 (10/550) ...
Generating image for seed 3875461 (11/550) ...
Generating image for seed 3875462 (12/550) ...
Generating image for seed 3875463 (

Let’s zip the generated files and download them.

In [ ]:
!zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

  adding: content/stylegan2/results/00000-generate-images/ (stored 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0025.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0014.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0007.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0018.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0017.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0010.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0015.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0024.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0021.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0004.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/log.txt (deflated 74%

#Interpolation


In [ ]:
!python run_generator.py generate-latent-walk --network=/content/stylegan2/results/00000-stylegan2-TF-shapesforms2-1gpu-config-f/network-snapshot-010000.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Local submit - run_dir: results/00002-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/stylegan2/results/00000-stylegan2-TF-shapesforms2-1gpu-config-f/network-snapshot-010000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
not enough values to generate walk
Generating image for step 0/204 ...
Generating image for step 1/204 ...
Generating image for step 2/204 ...
Generating image for step 3/204 ...
Generating image for step 4/204 ...
Generating image for step 5/204 ...
Generating image for step 6/204 ...
Generating image for step 7/204 ...
Generating image for step 8/204 ...
Generating image for step 9/204 ...
Generating image for step 10/204 ...
Generating image for step 11/204 ...
Generating image for step 12/204 ...
Generating image for step 13/204 ...
Generating image for step 14/204 ...


# How to convert the generated images to a video file
<p>looking into the generated images folder and reading all the png files there, specify the png file name by specifying the start of the number sequence for the generated files.</p>
<a href=https://stackoverflow.com/questions/24961127/how-to-create-a-video-from-images-with-ffmpeg>stackoverflow question about ffmpeg</a>

* can change the framerate to experiment

In [ ]:
#convert to video using the latent walk generated images (frames)
!ffmpeg -framerate 25 -start_number 00000 -i ./results/00002-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p latentwalk-test1-video1.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
#convert to video using the seed images -- this is not the one I want.
# !ffmpeg -framerate 25 -start_number 3875451 -i ./results/00001-generate-images/seed%07d.png -vcodec libx264 -pix_fmt yuv420p seed-video-2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# Copy files to mounted drive

In [ ]:
%pwd

'/content/stylegan2'

In [ ]:
%cp -a ./results/00002-generate-latent-walk ../drive/MyDrive/StyleGAN2-ADA/out